# Python GenAI Demo

This notebook demonstrates the Python wrapper for Chrome's built-in AI Prompt API.

**Requirements:**
- Chrome browser with Prompt API enabled
- Running in JupyterLab or Jupyter Notebook

In [ ]:
%pip install -q anywidget ipywidgets wiki3_ai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%gui asyncio

In [2]:
import anywidget
import traitlets


class LanguageModelWidget(anywidget.AnyWidget):
    _esm = """
    function render({ model, el }) {
      if (!('LanguageModel' in self)) {
        model.set("result", `Your browser doesn't support the Prompt API. If you're on Chrome, join the <a href="https://goo.gle/chrome-ai-dev-preview-join">Early Preview Program</a> to enable it.`);
      } else {
        model.set("result", "calling");
        self.LanguageModel.availability()
          .then(availability => { model.set("result", availability); model.save_changes(); })
          .catch(error => { model.set("result", error); model.save_changes(); });
      }
      model.on("change:count", () => {
        model.set("result", "checking");
        console.log("checking");
        self.LanguageModel.availability()
          .then(availability => { model.set("result", availability); model.save_changes(); })
          .catch(error => { model.set("result", error); model.save_changes(); });
      });
    }
	export default { render };
    """
    count = traitlets.Int(0).tag(sync=True)
    prompt = traitlets.Unicode("Foo").tag(sync=True)
    result = traitlets.Unicode("Bar").tag(sync=True)


counter = LanguageModelWidget()
counter.count = 42
counter

In [3]:
counter.count

42

In [4]:
counter.result

'available'

In [5]:
counter.count = 22

In [6]:
counter.result

'available'

In [ ]:
# Import the library
from wiki3_ai import LanguageModel, Availability, LanguageModelWidget

In [6]:
widget = LanguageModelWidget()
widget

In [6]:
widget.request

{}

In [6]:
options_dict = {}

params = {"options": options_dict}
result = await widget.send_request("availability", params)
result

Sending request ID:  fd35cacb-924b-41bb-818d-1121ce0c0111
Sent request:  {'id': 'fd35cacb-924b-41bb-818d-1121ce0c0111', 'method': 'availability', 'params': {'options': {}}}


'available'

In [7]:
import uuid

In [11]:
session_id = str(uuid.uuid4())

options_dict = {}

params = {"sessionId": session_id, "options": options_dict}
result = await widget.send_request("create", params)
result

Sending request ID:  3c3d6047-25e1-4243-a75f-a4293d253c9b
Sent request:  {'id': '3c3d6047-25e1-4243-a75f-a4293d253c9b', 'method': 'create', 'params': {'sessionId': 'e092192f-71a7-4c14-9468-e86369b9b106', 'options': {}}}


{'sessionId': 'e092192f-71a7-4c14-9468-e86369b9b106',
 'topK': 3,
 'temperature': 1,
 'inputUsage': 0,
 'inputQuota': 9216}

In [3]:
lm = LanguageModel()
lm.widget

In [4]:
session_id = await lm.create()
session_id

Sending request ID:  04882af1-4b3f-424d-a6f5-a2ff45743d65
Sent request:  {'id': '04882af1-4b3f-424d-a6f5-a2ff45743d65', 'method': 'create', 'params': {'sessionId': '24259487-6741-41e6-b4e8-6dc87dccd422', 'options': {}}}
Created session result:  {'sessionId': '24259487-6741-41e6-b4e8-6dc87dccd422', 'topK': 3, 'temperature': 1, 'inputUsage': 0, 'inputQuota': 9216}


'24259487-6741-41e6-b4e8-6dc87dccd422'

In [ ]:
print("foo", "bar")

## Check Availability

First, let's check if the Prompt API is available in your browser.

In [7]:
# Check if the API is available
availability = await lm.availability()
print(f"Model availability: {availability}")

if availability == Availability.UNAVAILABLE:
    print("\n❌ Chrome Prompt API is not available.")
    print("Make sure you're running in Chrome with the API enabled.")
elif availability == Availability.DOWNLOADING:
    print("\n⏳ Model is downloading. Please wait and try again.")
elif availability == Availability.AVAILABLE:
    print("\n✅ API is ready to use!")

Sending request ID:  e51cf041-55c1-4cfc-bb2e-9532702664ff
Sent request:  {'id': 'e51cf041-55c1-4cfc-bb2e-9532702664ff', 'method': 'availability', 'params': {'options': {}}}
Model availability: Availability.AVAILABLE

✅ API is ready to use!


## Get Model Parameters

Let's see what parameters the model supports.

In [13]:
params = await lm.params()
params

Sending request ID:  70d9b1af-0e74-4ed6-9ced-f67923f15cab
Sent request:  {'id': '70d9b1af-0e74-4ed6-9ced-f67923f15cab', 'method': 'params', 'params': {}}


Exception: DataCloneError: Failed to execute 'structuredClone' on 'Window': LanguageModelParams object could not be cloned.

In [12]:
if params:
    print(f"Default temperature: {params.default_temperature}")
    print(f"Max temperature: {params.max_temperature}")
    print(f"Default top-K: {params.default_top_k}")
    print(f"Max top-K: {params.max_top_k}")
else:
    print("Model parameters not available")

NameError: name 'params' is not defined

## Simple Prompt

Create a session and send a simple prompt.

In [ ]:
# Create a session
session_id = await lm.create()
session_id

In [8]:
# Send a prompt
result = await lm.prompt("Write a haiku about Python programming.")
result

Sending request ID:  0b648abf-090a-4f1a-afbf-67d59fb57701
Sent request:  {'id': '0b648abf-090a-4f1a-afbf-67d59fb57701', 'method': 'prompt', 'params': {'sessionId': '24259487-6741-41e6-b4e8-6dc87dccd422', 'input': 'Write a haiku about Python programming.', 'options': {}}}


'Clean, readable code,\nLogic flows, a gentle stream,\nWorlds built with Python. \n\n\n\n'

## Streaming Response

Stream the response as it's generated.

In [ ]:
print("Response: ", end="")
async for chunk in session.prompt_streaming("Tell me a short joke about computers."):
    print(chunk, end="", flush=True)
print()

## System Prompt

Use a system prompt to set the context for the conversation.

In [ ]:
# Create a session with a system prompt
assistant_session = await LanguageModel.create(
    {
        "initialPrompts": [
            {
                "role": "system",
                "content": "You are a helpful Python programming assistant who gives concise answers.",
            }
        ]
    }
)

# Ask a question
result = await assistant_session.prompt("How do I read a CSV file in Python?")
print(result)

## Multi-turn Conversation

Have a conversation with context maintained across prompts.

In [ ]:
# First message
result1 = await assistant_session.prompt("What is a list comprehension?")
print("Assistant:", result1)
print()

# Follow-up message (the assistant remembers the context)
result2 = await assistant_session.prompt("Can you show me an example?")
print("Assistant:", result2)

## Check Token Usage

Monitor how many tokens you've used.

In [ ]:
print(f"Current usage: {assistant_session.input_usage}/{assistant_session.input_quota} tokens")

# Measure how many tokens a prompt would use
usage = await assistant_session.measure_input_usage("What is machine learning?")
print(f"\nThis prompt would use approximately {usage} tokens")

## Structured Output

Use JSON schema to get structured responses.

In [ ]:
import json

# Define a JSON schema
schema = {
    "type": "object",
    "required": ["sentiment", "score"],
    "properties": {
        "sentiment": {"type": "string", "enum": ["positive", "negative", "neutral"]},
        "score": {"type": "number", "minimum": 0, "maximum": 1},
    },
}

# Get structured response
result = await session.prompt(
    "Analyze this review: The product exceeded all my expectations! Absolutely amazing!",
    {"responseConstraint": schema},
)

# Parse the JSON response
data = json.loads(result)
print(f"Sentiment: {data['sentiment']}")
print(f"Score: {data['score']}")

## Session Cloning

Clone a session to create different conversation branches.

In [ ]:
# Start a story
story_session = await LanguageModel.create(
    {"initialPrompts": [{"role": "system", "content": "You are a creative storyteller."}]}
)

await story_session.prompt("Once upon a time, there was a dragon.")

# Create two different story branches
branch1 = await story_session.clone()
branch2 = await story_session.clone()

result1 = await branch1.prompt("The dragon was friendly and helpful.")
print("Branch 1:", result1)
print()

result2 = await branch2.prompt("The dragon was fierce and terrifying.")
print("Branch 2:", result2)

## Cleanup

Destroy sessions when you're done to free up resources.

In [ ]:
await session.destroy()
await assistant_session.destroy()
await story_session.destroy()
await branch1.destroy()
await branch2.destroy()

print("✅ All sessions destroyed")